In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# import tensorflow.tensorflow_addons as tfa
import os
from tensorflow.keras.layers import Dense,BatchNormalization,LeakyReLU,Conv2DTranspose,Reshape,Conv2D,Dropout,Flatten,AveragePooling1D,Conv1D,MaxPooling1D
import tensorflow as tf
from tensorflow.keras.models import Sequential

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# 导入相关数据
os.chdir(r'D:\回归模型所需改进')
x_train = np.load('trainx.npy',allow_pickle=True).astype('float64').reshape(120,2500,1)
y_train = np.load('trainy.npy',allow_pickle=True).astype('float64').reshape(120,1)
x_test = np.load('testx.npy',allow_pickle=True).astype('float64').reshape(30,2500,1)
y_test = np.load('testy.npy',allow_pickle=True).astype('float64').reshape(30,1)
generated_x1 = np.load('XfromX1.npy', allow_pickle=True).astype('float64').reshape(120,2500,1)
generated_x2 = np.load('XfromX2.npy', allow_pickle=True).astype('float64').reshape(120,2500,1)
generated_x3 = np.load('XfromX3.npy', allow_pickle=True).astype('float64').reshape(120,2500,1)
generated_y1 = np.load('YfromY1.npy',allow_pickle = True).reshape(120,1).astype('float64')
generated_y2 = np.load('YfromY2.npy',allow_pickle = True).reshape(120,1).astype('float64')
generated_y3 = np.load('YfromY3.npy',allow_pickle = True).reshape(120,1).astype('float64')

In [ ]:
# 处理导入数据
# 自变量：X
comX1 = np.vstack([x_train,generated_x1]).reshape(240,2500,1)#R+A
comX2 = np.vstack([x_train,generated_x2]).reshape(240,2500,1)#R+B
comX3 = np.vstack([x_train,generated_x3]).reshape(240,2500,1)#R+C
comX4 = np.vstack([x_train,generated_x1,generated_x2]).reshape(360,2500,1)#R+A+B
comX5 = np.vstack([x_train,generated_x1,generated_x3]).reshape(360,2500,1)#R+A+C
comX6 = np.vstack([x_train,generated_x2,generated_x3]).reshape(360,2500,1)#R+B+C
comX7 = np.vstack([x_train,generated_x1,generated_x2,generated_x3]).reshape(480,2500,1)#R+A+B+C
# 因变量：y
comY1 = np.vstack([y_train,generated_y1]).reshape(240,1).astype('float64')#R+A
comY2 = np.vstack([y_train,generated_y2]).reshape(240,1).astype('float64')#R+B
comY3 = np.vstack([y_train,generated_y3]).reshape(240,1).astype('float64')#R+C
comY4 = np.vstack([y_train,generated_y1,generated_y2]).reshape(360,1).astype('float64')#R+A+B
comY5 = np.vstack([y_train,generated_y1,generated_y3]).reshape(360,1).astype('float64')#R+A+C
comY6 = np.vstack([y_train,generated_y2,generated_y3]).reshape(360,1).astype('float64')#R+B+C
comY7 = np.vstack([y_train,generated_y1,generated_y2,generated_y3]).reshape(480,1).astype('float64')#R+A+B+C

In [ ]:
# 增加多尺度
def Inception_ResNet():
    # 定义一个神经网络
    inputs = tf.keras.Input(shape=(2500,1))# 定义输入层
    
    c1 = tf.keras.layers.Conv1D(16,3, padding='same')(inputs)# 尺度一
    c1 = tf.keras.activations.gelu(c1)
    
    c2 = tf.keras.layers.Conv1D(16,3, padding='same')(c1)
    c2 = tf.keras.activations.gelu(c2)
    
    d1 = tf.keras.layers.Conv1D(16,5, padding='same')(inputs)# 尺度二
    d1 = tf.keras.activations.gelu(d1)
    
    d2 = tf.keras.layers.Conv1D(16,5, padding='same')(d1)
    d2 = tf.keras.activations.gelu(d2)
    
    add1 = tf.keras.layers.add([c1, d1])
    p1 = tf.keras.layers.MaxPool1D(3)(add1)# 最大池化层
    
    c3 = tf.keras.layers.Conv1D(64,5, padding='same')(p1)
    c3 = tf.keras.activations.gelu(c3)
    
    c4 = tf.keras.layers.Conv1D(64,5, padding='same')(c3)
    c4 = tf.keras.activations.gelu(c4)
    
    d3 = tf.keras.layers.Conv1D(64,3, padding='same')(p1)
    d3 = tf.keras.activations.gelu(d3)
    
    d4 = tf.keras.layers.Conv1D(64,3, padding='same')(d3)
    d4 = tf.keras.activations.gelu(d4)
    
    add2 = tf.keras.layers.add([c4, d4])
    p2 = tf.keras.layers.MaxPool1D(3)(add2)
    
    c5 = tf.keras.layers.Conv1D(128,3, padding='same')(p2)
    c5 = tf.keras.activations.gelu(c5)
    
    c6 = tf.keras.layers.Conv1D(128,3, padding='same')(c5)
    c6 = tf.keras.activations.gelu(c6)
    
    d5 = tf.keras.layers.Conv1D(128,5, padding='same')(p2)
    d5 = tf.keras.activations.gelu(d5)
    
    d6 = tf.keras.layers.Conv1D(128,5, padding='same')(d5)
    d6 = tf.keras.activations.gelu(d6)
    
    add3 = tf.keras.layers.add([c6, d6])
    p3 = tf.keras.layers.MaxPool1D(3)(add3)
    
    c7 = tf.keras.layers.Conv1D(16,5, padding='same')(p3)
    c7 = tf.keras.activations.gelu(c7)
    
    c8 = tf.keras.layers.Conv1D(16,5, padding='same')(c7)
    c8 = tf.keras.activations.gelu(c8)
    
    d7 = tf.keras.layers.Conv1D(16,3, padding='same')(p3)
    d7 = tf.keras.activations.gelu(d7)
    
    d8 = tf.keras.layers.Conv1D(16,3, padding='same')(d7)
    d8 = tf.keras.activations.gelu(d8)
    
    add4 = tf.keras.layers.add([c8, d8])
    p4 = tf.keras.layers.MaxPool1D(5)(add4)
    
    f1 = tf.keras.layers.Flatten()(p4)# 拉伸层
    output = tf.keras.layers.Dense(1, activation='linear')(dropout)# 全连接层
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model 
model = Inception_ResNet()
model.summary()

In [ ]:
# 自定义损失函数
def MSE(x_batch, y_batch):# 输入的真实x，y
    y_pred = model(x_batch)
    y_pred = tf.cast(y_pred,dtype=tf.float64)
    y_batch = tf.cast(y_batch,dtype=tf.float64)
    mse_loss = tf.reduce_mean(tf.square(y_pred-y_batch))
    mse_loss = tf.cast(mse_loss,dtype=tf.float64)
    return mse_loss
def MSE1(x_batch, y_batch):# 输入的真实x，y
    y_pred = model(x_batch, training=False)
    y_pred = tf.cast(y_pred,dtype=tf.float64)
    y_batch = tf.cast(y_batch,dtype=tf.float64)
    mse_loss = tf.reduce_mean(tf.square(y_pred-y_batch))
    mse_loss = tf.cast(mse_loss,dtype=tf.float64)
    return mse_loss
def r_2(x_batch, y_batch):
    y_pred = model(x_batch)
    y_pred = tf.cast(y_pred,dtype=tf.float64)
    y_batch = tf.cast(y_batch,dtype=tf.float64)
    y_mean = tf.reduce_mean(y_batch)
    SSE = tf.reduce_sum(tf.square(y_batch-y_pred))    
    SST = tf.reduce_sum(tf.square(y_batch-y_mean))  
    R2 = 1 - SSE/SST
    return R2
model_optimizer1 = tf.optimizers.Adam(0.001)
model_optimizer2 = tf.optimizers.Adam(0.0005)
model_optimizer2 = tf.optimizers.Adam(0.00025)

In [ ]:
# 定义早停法一
def stop_train(patience, score, epoch, num_patience,i):
    global best_score
    if epoch == 0:# 记录最好的结果
        best_score = score
        print('更新了历史最优MSE为:%s'%best_score)
    elif score >= best_score:# 如果模型没有进步的话,这次的MSE应该高于历史最低MSE
        #将迭代次数增1
        num_patience = num_patience+1
        print('MSE:容忍度更新为：%s'%num_patience)
        print("此次训练未成功优化性能，历史最优MSE仍为：%s"%best_score)
        print("此次训练未成功优化性能，本次MSE为：%s"%score)
        if num_patience >= patience:# 大于容忍度，模型训练暂停
            return -1
    else:
        best_score = score# 更新历史最优MSE
        num_patience = 0 # 重置容忍度
        print("//**********************************//")
        print('更新了历史最优MSE为:%s'%best_score)
    model.save('model{}.h5'.format(i))
    return num_patience
# 定义早停法二
def stop_train_r2(patience, input_r2, epoch, r2_patience,i):
    global best_r2
    if epoch == 0:# 记录最好的结果
        best_r2 = input_r2
        print('更新了历史最优测试集R2为:%s'%best_score)
    elif input_r2 <= best_r2:# 如果模型没有进步的话,这次的MSE应该高于历史最低MSE
        #将迭代次数增1
        r2_patience = r2_patience+1
        print('R2:容忍度更新为：%s'%r2_patience)
        print("此次训练未成功优化性能，历史最优测试集R2仍为：%s"%best_r2)
        print("此次训练未成功优化性能，本次测试集R2为：%s"%input_r2)
        if r2_patience >= patience:# 大于容忍度，模型训练暂停
            return -1
    else:
        best_r2 = input_r2# 更新历史最优MSE
        r2_patience = 0 # 重置容忍度
        print("//**********************************//")
        print('更新了历史最优测试集R2为:%s'%best_r2)
    return r2_patience

In [ ]:
dataSetX = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(80).batch(40)
dataSetXA = tf.data.Dataset.from_tensor_slices((comX1, comY1)).shuffle(160).batch(80)
dataSetXB = tf.data.Dataset.from_tensor_slices((comX2, comY2)).shuffle(160).batch(80)
dataSetXC = tf.data.Dataset.from_tensor_slices((comX3, comY3)).shuffle(160).batch(80)
dataSetXAB = tf.data.Dataset.from_tensor_slices((comX4, comY4)).shuffle(240).batch(120)
dataSetXAC = tf.data.Dataset.from_tensor_slices((comX5, comY5)).shuffle(240).batch(120)
dataSetXBC = tf.data.Dataset.from_tensor_slices((comX6, comY6)).shuffle(240).batch(120)
dataSetXABC = tf.data.Dataset.from_tensor_slices((comX7, comY7)).shuffle(320).batch(160)

In [ ]:
test_r2 = []
train_r2 = []
def train(dataset, epochs, trainxFor_thisepoch, trainyFor_thisepoch):
    i = 0
    num_patience = 0
    r2_patience = 0
    for epoch in range(epochs):
        if num_patience == -1 or r2_patience == -1:
            break
        if epoch < 400:
            for xBatch, yBatch in dataset:
                with tf.GradientTape() as regressionTape:
                    loss = MSE(xBatch, yBatch)
                gradients_of_regression = regressionTape.gradient(loss, model.trainable_variables)
                model_optimizer1.apply_gradients(zip(gradients_of_regression, model.trainable_variables))# 更新权值
                loss = MSE1(xBatch, yBatch)# 这里设置的应该是训练集整体
                score = loss
                r1 = r_2(trainxFor_thisepoch, trainyFor_thisepoch)# 这里设置的应该是训练集整体
                r2 = r_2(x_test, y_test)# 这里设置的应该是训练集整体 
                test_r2.append(r2)
                train_r2.append(r1)
                num_patience = stop_train(300, score, epoch, num_patience,i)
                r2_patience = stop_train_r2(300, r2, epoch, r2_patience,i)
                i = i+1
            if num_patience == -1 or r2_patience == -1:
                break
        if 800>epoch >=400:
            if epoch==400:
                num_patience = 0
                r2_patience = 0
            for xBatch, yBatch in dataset:
                with tf.GradientTape() as regressionTape:
                    loss = MSE(xBatch, yBatch)
                gradients_of_regression = regressionTape.gradient(loss, model.trainable_variables)
                model_optimizer2.apply_gradients(zip(gradients_of_regression, model.trainable_variables))# 更新权值(更换学习率)
                loss = MSE1(xBatch, yBatch)# 这里设置的应该是训练集整体
                score = loss
                r1 = r_2(trainxFor_thisepoch, trainyFor_thisepoch)# 这里设置的应该是训练集整体
                r2 = r_2(x_test, y_test)# 这里设置的应该是训练集整体 
                test_r2.append(r2)
                train_r2.append(r1)
                num_patience = stop_train(300, score, epoch, num_patience,i)
                r2_patience = stop_train_r2(300, r2, epoch, r2_patience,i)
                i = i+1
            if num_patience == -1 or r2_patience == -1:
                break
        if epoch >=800:
            if epoch==800:
                num_patience = 0
                r2_patience = 0
            for xBatch, yBatch in dataset:
                with tf.GradientTape() as regressionTape:
                    loss = MSE(xBatch, yBatch)
                gradients_of_regression = regressionTape.gradient(loss, model.trainable_variables)
                model_optimizer3.apply_gradients(zip(gradients_of_regression, model.trainable_variables))# 更新权值(更换学习率)
                loss = MSE1(xBatch, yBatch)# 这里设置的应该是训练集整体
                score = loss
                r1 = r_2(trainxFor_thisepoch, trainyFor_thisepoch)# 这里设置的应该是训练集整体
                r2 = r_2(x_test, y_test)# 这里设置的应该是训练集整体 
                test_r2.append(r2)
                train_r2.append(r1)
                num_patience = stop_train(120, score, epoch, num_patience,i)
                r2_patience = stop_train_r2(120, r2, epoch, r2_patience,i)
                i = i+1
            if num_patience == -1 or r2_patience == -1:
                break
        print('这是第%s次训练'%epoch)

In [ ]:
import time
# 模型训练 
#记录运行时间
os.chdir(r'D:/Inception/X')# 需要更改
with tf.device('/GPU:0'):
    start = time.perf_counter()
    train(dataSetX, 4000, x_train, y_train)# 需要更改
    end = time.perf_counter()
    print('Running time: %s Seconds'%(end-start))

In [ ]:
test_r2 = np.array(test_r2)
train_r2 = np.array(train_r2)

In [ ]:
np.save("Test_r2.npy",test_r2)
np.save("Train_r2.npy",train_r2)